In [ ]:
#| default_exp smoothing

## Mesh smoothing

> Smooth mesh with various filters

In [ ]:
#| export

from blender_tissue_cartography import mesh as tcmesh
from blender_tissue_cartography import registration as tcreg

import numpy as np
from copy import deepcopy
import warnings
import igl

from scipy import sparse

In [ ]:
#| hide
from blender_tissue_cartography import interpolation as tcinterp
from skimage import transform
import matplotlib.pyplot as plt

In [ ]:
#| hide
np.set_printoptions(suppress=True)

### Smoothing in `libigl`

We implement Laplacian and Taubin smoothing for vertex positions using `libigl`'s Laplacian operator.

In [ ]:
#| export

def get_uniform_laplacian(tris, normalize=True):
    """
    Get uniform Laplacian (purely connectivity-based) as a sparse matrix.
    
    If normalize, the diagonal = -1. Else, the diagonal equals the number of neighbors.
    """
    a = igl.adjacency_matrix(tris)
    a_sum = np.squeeze(np.asarray(a.sum(axis=1)))
    a_diag = sparse.diags(a_sum)
    if normalize:
        return sparse.diags(1/a_sum)@(a - a_diag)
    return (a - a_diag)

In [ ]:
#| export

def smooth_laplacian(mesh: tcmesh.ObjMesh, lamb=0.5, n_iter=10, method="explicit", boundary="fixed") -> tcmesh.ObjMesh:
    """
    Smooth mesh vertex positions using Laplacian filter.
    
    Assumes mesh is triangular.
    
    Parameters
    ----------
    mesh : ObjMesh
        Initial mesh.
    lamb : float, default 0.5
        Filter strength. Higher = more smoothing.
    n_iter : int
        Filter iterations
    method : str, default "explicit"
        Can use "explicit" (fast, simple) or "implicit" (slow, more accurate) methods.
    boundary : str, "fixed" or "free"
        Whether to allow mesh boundaries to move

    Returns
    -------
    mesh_smoothed : ObjMesh
        Smoothed mesh.

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular - result may be incorrect", RuntimeWarning)
    v_smoothed = np.copy(mesh.vertices)
    f = mesh.tris
    boundary_vertices = igl.boundary_facets(f)[:, 0]

    if method == "implicit":
        laplacian = igl.cotmatrix(v_smoothed, f)
        for _ in range(n_iter):
            mass = igl.massmatrix(v_smoothed, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
            v_smoothed = sparse.linalg.spsolve(mass - lamb * laplacian, mass.dot(v_smoothed))
            if boundary == "fixed":
                v_smoothed[boundary_vertices] = mesh.vertices[boundary_vertices]
    elif method == "explicit":
        laplacian_uniform = get_uniform_laplacian(f)
        for _ in range(n_iter):
            v_smoothed += lamb*laplacian_uniform.dot(v_smoothed)
            if boundary == "fixed":
                v_smoothed[boundary_vertices] = mesh.vertices[boundary_vertices]
    mesh_smoothed = tcmesh.ObjMesh(v_smoothed, mesh.faces, texture_vertices=mesh.texture_vertices,
                                 normals=None, name=mesh.name)
    mesh_smoothed.set_normals()
    return mesh_smoothed

In [ ]:
# Load a test mesh
mesh_registered = tcmesh.ObjMesh.read_obj(f"datasets/wrapping_example/Drosophila_reference_registered.obj")

  o Drosophila_reference_preregistered


In [ ]:
mesh_smoothed = smooth_laplacian(mesh_registered, lamb=0.5, n_iter=10, method="explicit")
mesh_smoothed.write_obj("datasets/wrapping_example/Drosophila_reference_smoothed_uniform_igl.obj")

In [ ]:
boundary_vertices = igl.boundary_facets(mesh_registered.texture_tris)[:, 0]

In [ ]:
np.allclose(mesh_smoothed.texture_vertices[boundary_vertices],
            mesh_registered.texture_vertices[boundary_vertices])

True

In [ ]:
#| export

def smooth_taubin(mesh: tcmesh.ObjMesh, lamb=0.5, nu=0.53, n_iter=10,) -> tcmesh.ObjMesh:
    """
    Smooth using Taubin filter (like Laplacian, but avoids shrinkage).
    
    Assumes mesh is triangular. See "Improved Laplacian Smoothing of Noisy Surface Meshes"
    J. Vollmer, R. Mencl, and H. Muller.
    
    Parameters
    ----------
    mesh : ObjMesh
        Initial mesh.
    lamb : float, default 0.5
        Filter strength. Higher = more smoothing.
    nu : float, default 0.53
        Counteract shrinkage. Higher = more dilation.
    n_iter : int
        Filter iterations

    Returns
    -------
    mesh_smoothed : ObjMesh
        Smoothed mesh.

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular - result may be incorrect", RuntimeWarning)
    v_smoothed = np.copy(mesh.vertices)
    laplacian_uniform = get_uniform_laplacian(mesh.tris)
    for _ in range(n_iter):
        v_smoothed += lamb*laplacian_uniform.dot(v_smoothed)
        v_smoothed -= nu*laplacian_uniform.dot(v_smoothed)
    mesh_smoothed = tcmesh.ObjMesh(v_smoothed, mesh.faces, texture_vertices=mesh.texture_vertices,
                                 normals=None, name=mesh.name)
    mesh_smoothed.set_normals()
    return mesh_smoothed

In [ ]:
mesh_smoothed_taubin = smooth_taubin(mesh_registered, lamb=0.5, nu=0.53, n_iter=10)
mesh_smoothed_taubin.write_obj("datasets/wrapping_example/Drosophila_reference_smoothed_taubin_igl.obj")

## Texture smoothing

Sometimes, UV maps can become very deformed, or even display self-intersection. Smoothing can fix this.

In [ ]:
#| export

def smooth_laplacian_texture(mesh: tcmesh.ObjMesh, lamb=0.5, n_iter=10, boundary="fixed") -> tcmesh.ObjMesh:
    """
    Smooth mesh texture positions using Laplacian filter.
    
    This function is very helpful in fixing UV maps with flipped triangles, as detected by
    igl.flipped_triangles. Assumes mesh is triangular.
    
    Parameters
    ----------
    mesh : ObjMesh
        Initial mesh.
    lamb : float, default 0.5
        Filter strength. Higher = more smoothing.
    n_iter : int
        Filter iterations
    boundary : str, "fixed" or "free"
         Whether to allow UV "island" boundary to move

    Returns
    -------
    mesh_smoothed : ObjMesh
        Smoothed mesh.

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular - result may be incorrect", RuntimeWarning)
    v_smoothed = np.copy(mesh.texture_vertices)
    f = mesh.texture_tris
    laplacian_uniform = get_uniform_laplacian(f)
    boundary_vertices = igl.boundary_facets(f)[:, 0]

    for _ in range(n_iter):
        v_smoothed += lamb*laplacian_uniform.dot(v_smoothed)
        if boundary == "fixed":
            v_smoothed[boundary_vertices] = mesh.texture_vertices[boundary_vertices]
    mesh_smoothed = tcmesh.ObjMesh(mesh.vertices, mesh.faces, texture_vertices=v_smoothed,
                                 normals=mesh.normals, name=mesh.name)
    return mesh_smoothed

In [ ]:
mesh_texture_smoothed = smooth_laplacian_texture(mesh_registered, lamb=0.5, n_iter=10,)


### On-surface smoothing

Smooth, but project at each step so the mesh vertices are back on the surface. This is very useful to smooth out surface-surface maps.

In [ ]:
#| export

def smooth_laplacian_on_surface(mesh: tcmesh.ObjMesh, n_iter=10, lamb=0.5, n_iter_laplace=10,
                                boundary="fixed") -> tcmesh.ObjMesh:
    """
    Smooth mesh vertex positions using Laplacian filter and project vertices back to the original surface.
    
    Alternates between Laplacian smoothing and projecting back to the original surface. Uses
    explicit method for Laplacian smoothing
    
    Parameters
    ----------
    mesh : ObjMesh
        Initial mesh.
    n_iter : int
        Number of iterations at each step
    lamb : float, default 0.5
        Filter strength. Higher = more smoothing.
    n_iter_laplace : int
        Laplace filter iterations. If reprojection messes up your mesh, decrease this number.
    boundary : str, "fixed" or "free"
        Whether to allow mesh boundaries to move

    Returns
    -------
    mesh_smoothed : ObjMesh
        Smoothed mesh.

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular - result may be incorrect", RuntimeWarning)
    v_reference = np.copy(mesh.vertices)
    v_smoothed = np.copy(mesh.vertices)
    f = mesh.tris
    boundary_vertices = igl.boundary_facets(f)[:, 0]
    laplacian_uniform = get_uniform_laplacian(f)

    for _ in range(n_iter):
        # smooth
        for _ in range(n_iter):
            v_smoothed += lamb*laplacian_uniform.dot(v_smoothed)
            if boundary == "fixed":
                v_smoothed[boundary_vertices] = mesh.vertices[boundary_vertices]
        # project 
        _, _, v_smoothed = igl.point_mesh_squared_distance(v_smoothed, v_reference, f)
        v_reference = np.copy(v_smoothed)
    mesh_smoothed = tcmesh.ObjMesh(v_smoothed, mesh.faces, texture_vertices=mesh.texture_vertices,
                                 normals=mesh.normals, name=mesh.name)
    mesh_smoothed.set_normals()
    return mesh_smoothed

In [ ]:
mesh_test = tcmesh.ObjMesh.read_obj(f"datasets/movie_example/meshes_wrapped_reverse/mesh_01_wrapped_reverse.obj")
mesh_smoothed = smooth_laplacian_on_surface(mesh_test, n_iter=20, lamb=0.5, n_iter_laplace=5)
mesh_smoothed.write_obj(f"datasets/movie_example/on_surface_smooth.obj")